In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%matplotlib inline

In [3]:
import keras
import rl
import gym
import numpy
import skimage.io
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
class Test(gym.Env):
    
    def __init__(self, neurons, neuron_updates, weight_updates):
        self.neurons = neurons
        self.neuron_updates = neuron_updates
        self.weight_updates = weight_updates
        self.action_space = gym.spaces.Dict({
            "neurons": gym.spaces.MultiDiscrete(neuron_updates * [neurons]),
            "weights": gym.spaces.Tuple(neuron_updates * (gym.spaces.MultiDiscrete(weight_updates * [neurons]),)),
            "deltas": gym.spaces.Box(-numpy.inf, numpy.inf, (neuron_updates, weight_updates), dtype=float)
        })
        self.observation_space = gym.spaces.Box(-1, 1, (neurons, neurons), dtype=float)
        self.state = numpy.zeros((neurons, neurons), dtype=float)
        self.random_seed = None
    
    def seed(self, seed):
        self.random_seed = seed
        numpy.random.seed(seed)
        return seed
    
    def close(self):
        pass
    
    def reset(self):
        self.state[:] = 0
        return self.state
    
    def step(self, action):
        for n in range(self.neuron_updates):
            for w in range(self.weight_updates):
                self.state[action["neurons"][n], action["weights"][n, w]] += action["deltas"][n, w]
        reward = numpy.linalg.det(self.state)
        terminal = reward < 0
        return self.state, reward, terminal, {}

In [ ]:
N = 10
NU = 10
WU = 10

In [ ]:
test = Test(N, NU, WU)

In [ ]:
test.reset()

In [ ]:
for i in range(1):
    action = {
        "neurons": numpy.random.randint(0, N, NU),
        "weights": numpy.random.randint(0, N, (NU, WU)),
        "deltas": numpy.random.uniform(-1, 1, (NU, WU))
    }
    state, reward, terminal, _ = test.step(action)
    plt.figure()
    plt.imshow(state)
    print(reward)
    if terminal:
        test.reset()

In [ ]:
# state_input = keras.layers.Input((N * N,))
# x = keras.layers.Dense(N * N, activation='relu')(state_input)
# x = keras.layers.Dense(N * N, activation='relu')(x)
# x = keras.layers.Dense(N * N, activation='relu')(x)
# weight_output = keras.layers.Dense(N * N, activation='linear')(x)
# weight_model = keras.models.Model(state_input, weight_output)
# weight_model.summary()

In [4]:
def build_choice_model(neurons):
    input_layer = keras.layers.Input((neurons ** 2,), name='state_input')
    x = keras.layers.Dense(neurons ** 2, activation='relu')(input_layer)
    x = keras.layers.Dense(neurons ** 2, activation='relu')(x)
    x = keras.layers.Dense(neurons ** 2, activation='relu')(x)
    choice_output = keras.layers.Dense(neurons ** 2, activation='linear', name='choice_output')(x)
    choice_model = keras.models.Model(input_layer, choice_output, name='choice_model')
    return choice_model

In [5]:
def build_delta_model(neurons):
    input_layers = [keras.layers.Input((neurons ** 2,), name='state_input'), keras.layers.Input((neurons ** 2,), name='choice_output')]
    x = keras.layers.Multiply()(input_layers)
    x = keras.layers.Dense(neurons ** 2, activation='relu')(x)
    x = keras.layers.Dense(neurons ** 2, activation='relu')(x)
    x = keras.layers.Dense(neurons ** 2, activation='relu')(x)
    delta_output = keras.layers.Dense(1, activation='linear', name='delta_output')(x)
    delta_model = keras.models.Model(input_layers, delta_output, name='delta_model')
    return delta_model

In [6]:
N = 10
U = 1

In [19]:
choice_model = build_choice_model(N)
delta_model = build_delta_model(N)
delta_tensor = delta_model([choice_model.input, choice_model.output])
combined_model = keras.models.Model(choice_model.input, [choice_model.output, delta_tensor])

In [ ]:
# state_input = keras.layers.Input((N ** 2,), name='state_input')
# choice_tensor = build_choice_model(N)(state_input)
# choice_model = keras.models.Model(state_input, choice_tensor)
# delta_model = build_delta_model(N)
# delta_tensor = delta_model([state_input, choice_model.output])
# combined_model = keras.models.Model(choice_model.input, delta_tensor)

In [20]:
choice_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
state_input (InputLayer)     (None, 100)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_38 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_39 (Dense)             (None, 100)               10100     
_________________________________________________________________
choice_output (Dense)        (None, 100)               10100     
Total params: 40,400
Trainable params: 40,400
Non-trainable params: 0
_________________________________________________________________


In [21]:
delta_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
state_input (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
choice_output (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
multiply_7 (Multiply)           (None, 100)          0           state_input[0][0]                
                                                                 choice_output[0][0]              
__________________________________________________________________________________________________
dense_40 (Dense)                (None, 100)          10100       multiply_7[0][0]                 
__________

In [22]:
combined_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
state_input (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
dense_37 (Dense)                (None, 100)          10100       state_input[0][0]                
__________________________________________________________________________________________________
dense_38 (Dense)                (None, 100)          10100       dense_37[0][0]                   
__________________________________________________________________________________________________
dense_39 (Dense)                (None, 100)          10100       dense_38[0][0]                   
__________________________________________________________________________________________________
choice_out

In [ ]:
# choice_layers = keras.models.Sequential([
#     keras.layers.Dense(N ** 2, activation='relu', input_shape=(N ** 2,)),
#     keras.layers.Dense(N ** 2, activation='relu'),
#     keras.layers.Dense(N ** 2, activation='relu'),
#     keras.layers.Dense(N ** 2, activation='linear', name='choice_output')
# ])

In [ ]:
# delta_layers = keras.models.Sequential([
#     keras.layers.Concatenate(),
#     keras.layers.Dense(neurons ** 2, activation='relu')(x),
#     keras.layers.Dense(neurons ** 2, activation='relu')(x),
#     keras.layers.Dense(neurons ** 2, activation='relu')(x),
#     keras.layers.Dense(1, activation='linear', name='delta_output')(x)
# ])

In [ ]:
# state_input = keras.layers.Input((N,), name='state_input')
# choice_output = build_choice_model(N)
# choice_model = keras.models.Model(state_input, choice_output, name='choice_model')
# choice_input = keras.layers.Input((N ** 2,), name='choice_input')
# delta_output_standalone = build_delta_model([state_input, choice_input], N)
